<a id="sectionx"></a>
# Consumer Finance - Demo - Helper Calls
* [Create New Customer](#section0)
* [Add New Card Transaction](#section1)
* [Get Accounts](#section3)
* [Disburse Loan](#section4)
* [Create ALL in 1 go](#section2)


GitHub: [https://github.com/MkershMambu/MambuAPINotebook/blob/master/Consumer%20Finance%20-%20Debit%20Credit%20Account.ipynb](https://github.com/MkershMambu/MambuAPINotebook/blob/master/Consumer%20Finance%20-%20Debit%20Credit%20Account.ipynb)

Viewer: [https://nbviewer.jupyter.org/github/MkershMambu/MambuAPINotebook/](https://nbviewer.jupyter.org/github/MkershMambu/MambuAPINotebook/blob/master/Consumer%20Finance%20-%20Debit%20Credit%20Account.ipynb)




In [5]:
%run API.py
%run ENV.py
from IPython.core.display import HTML
setENV(ENV)

<a id="section0"></a>
[back](#sectionx)
## Create New Customer
* Create a New Customer
* Add a Credit Arrangement to the Customer

In [6]:
%%writefile NEWCUSTTEMP
{
"firstName": "{{FirstName}}",
"lastName": "{{LastName}}",
"preferredLanguage": "ENGLISH",
"addresses": [
    {
      "country": "UK",
      "city": "Liverpool"
    }
],
"notes": "Some Notes on this person",
"gender": "MALE",
  
"assignedBranchKey": "{{BranchID}}"

}

Overwriting NEWCUSTTEMP


In [7]:
def createNewCustomerTest(fname,lname):
    createNewCustomer(fname,lname, "8a818f156ccf5fb1016cd2e8e4532b09")
def createNewCustomerDemo(fname,lname):
    createNewCustomer(fname,lname, "8a818f5f6cbe6621016cbf217c9e5060")    
    
def createNewCustomer(fname,lname, branchID):
    global newCustomerID, custShortID
    PLACEHOLDERS = {
        "BranchID": branchID,
        "FirstName": fname,
        "LastName": lname
    }
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    r = POST('{{env4}}/clients', headers=HEADERS, params=PARAMS, body="NEWCUSTTEMP", bodyparts=PLACEHOLDERS)
    print ("API status:")
    print(r.status_code)
    print("Now the JSON:")
    print(r.text)

    # Remember the ID of the customer created we will use it below 
    newCustomerID = r.json()['encodedKey']
    custShortID = r.json()['id']
   

In [8]:
createNewCustomerTest("Harry", "Styles")

API status:
201
Now the JSON:
{"encodedKey":"8a818f517618f4f401761a1ee28d4fec","id":"542266132","state":"INACTIVE","creationDate":"2020-11-30T18:06:40+01:00","lastModifiedDate":"2020-11-30T18:06:40+01:00","approvedDate":"2020-11-30T18:06:39+01:00","firstName":"Harry","lastName":"Styles","preferredLanguage":"ENGLISH","gender":"MALE","notes":"Some Notes on this person","assignedBranchKey":"8a818f156ccf5fb1016cd2e8e4532b09","clientRoleKey":"8a818e74677a2e9201677ec2b4c336aa","loanCycle":0,"groupLoanCycle":0,"addresses":[{"encodedKey":"8a818f517618f4f401761a1ee28d4fed","parentKey":"8a818f517618f4f401761a1ee28d4fec","city":"Liverpool","country":"UK","indexInList":0}],"idDocuments":[]}


In [4]:
%%writefile NEWCREDITARRANGEMENT
{
    "amount": 10000.0,
    "availableCreditAmount": 10000.0,
    "expireDate": "2030-08-23T00:00:00+02:00",
    "exposureLimitType": "OUTSTANDING_AMOUNT",
    "holderKey": "{{CustomerID}}",
    "holderType": "CLIENT",  
    "notes": "",
    "startDate": "2019-08-23T00:00:00+02:00",
    "state": "APPROVED",
    "_URepayOptions": {
            "AutoRepayMethod": "Direct-Debit",
            "PaymentDueDay": "1",
            "ShortMonthOption": "late"
        }
}

Overwriting NEWCREDITARRANGEMENT


In [5]:
def createNewCreditArrangement():
    global CAID
    PLACEHOLDERS = {
        "CustomerID": newCustomerID
    }
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    r = POST('{{env4}}/creditarrangements', headers=HEADERS, params=PARAMS, body="NEWCREDITARRANGEMENT", bodyparts=PLACEHOLDERS)
    print ("API status:")
    print(r.status_code)
    print("Now the JSON:")
    print(r.text)
    CAID = r.json()['id']

In [235]:
createNewCreditArrangement()

API status:
201
Now the JSON:
{"encodedKey":"8a818f796cd39234016cd3d082dd0384","id":"ZUG055","holderType":"CLIENT","holderKey":"8a818f746cd2fea7016cd3ace4c74d28","amount":10000.0,"availableCreditAmount":10000.0,"consumedCreditAmount":0.0,"state":"APPROVED","creationDate":"2019-08-27T18:02:33+02:00","lastModifiedDate":"2019-08-27T18:02:33+02:00","startDate":"2019-08-23T00:00:00+02:00","expireDate":"2020-08-23T00:00:00+02:00","approvedDate":"2019-08-27T18:02:33+02:00","exposureLimitType":"OUTSTANDING_AMOUNT","notes":"","_URepayOptions":{"ShortMonthOption":"late","PaymentDueDay":"1","AutoRepayMethod":"Direct-Debit"}}


### Create Accounts for the Customer

In [6]:
%%writefile CURACCTEMP
{
  "overdraftInterestSettings": {
        "interestRateSettings": {
            "encodedKey": "8a818f9f6cc14d6e016cc2fcdbef4923",
            "interestChargeFrequency": "ANNUALIZED",
            "interestChargeFrequencyCount": 1,
            "interestRateReviewCount": 31,
            "interestRateReviewUnit": "DAYS",
            "interestRateSource": "INDEX_INTEREST_RATE",
            "interestRateTerms": "FIXED",
            "interestSpread": 0.0
        }
    },
    "overdraftSettings": {
        "allowOverdraft": true,
        "overdraftExpiryDate": "2020-05-02T00:00:00+02:00",
        "overdraftLimit": 0.0
    },
  
  "accountType": "CURRENT_ACCOUNT",
  "name": "Current Account (with overdraft)",
  "accountHolderKey": "{{CustomerID}}",
  "productTypeKey": "{{ProductKey}}",
  "currencyCode": "EUR",
  "accountHolderType": "CLIENT"
}


Overwriting CURACCTEMP


In [7]:
%%writefile DDAACCTEMP
{
  "overdraftInterestSettings": {
        "interestRateSettings": {
            "encodedKey": "8a818f9c6cd48156016cd6ef16ec3c25",
            "interestChargeFrequency": "ANNUALIZED",
            "interestChargeFrequencyCount": 1,
            "interestRate": 0.0,
            "interestRateSource": "FIXED_INTEREST_RATE",
            "interestRateTerms": "FIXED",
            "interestRateTiers": []
        }
    },
    "overdraftSettings": {
        "allowOverdraft": true,
        "overdraftExpiryDate": "2020-05-02T00:00:00+02:00",
        "overdraftLimit": 0.0
    },
  
  "accountType": "CURRENT_ACCOUNT",
  "name": "TEMP Card Account",
  "accountHolderKey": "{{CustomerID}}",
  "productTypeKey": "{{ProductKey}}",
  "currencyCode": "EUR",
  "accountHolderType": "CLIENT"
}

Overwriting DDAACCTEMP


In [8]:
def createCurrentAccount():
    createDepositAccount("8a818f5f6cbe6621016cbf4f2db95756", "CURACCTEMP")

def createTEMPCardDDAAccount():
    createDepositAccount("8a818f5f6cbe6621016cbf7310ff6064", "DDAACCTEMP")
    
def createDepositAccount(productKey, tempFile):
    global CurrentAccID
    PLACEHOLDERS = {
        "CustomerID": newCustomerID,
        "ProductKey": productKey
    }
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    r = POST('{{env4}}/deposits', headers=HEADERS, params=PARAMS, body=tempFile, bodyparts=PLACEHOLDERS)
    print ("API status:")
    print(r.status_code)
    print("Now the JSON:")
    print(r.text)
    CurrentAccID = r.json()['id']


In [421]:
#createTEMPCardDDAAccount()
createCurrentAccount()

API status:
201
Now the JSON:
{"encodedKey":"8a818ea06cd4910e016cd6f80df239a6","creationDate":"2019-08-28T08:45:43+02:00","lastModifiedDate":"2019-08-28T08:45:43+02:00","id":"SAUU325","name":"Current Account (with overdraft)","accountHolderType":"CLIENT","accountHolderKey":"8a818efc6cd4813e016cd4ab3f3600b9","accountState":"PENDING_APPROVAL","productTypeKey":"8a818f5f6cbe6621016cbf4f2db95756","accountType":"CURRENT_ACCOUNT","currencyCode":"EUR","assignedBranchKey":"8a818f156ccf5fb1016cd2e8e4532b09","internalControls":{},"overdraftSettings":{"allowOverdraft":true,"overdraftLimit":0.0,"overdraftExpiryDate":"2020-05-02T00:00:00+02:00"},"interestSettings":{"interestPaymentSettings":{"interestPaymentDates":[]}},"overdraftInterestSettings":{"interestRateSettings":{"encodedKey":"8a818ea06cd4910e016cd6f80df239a7","interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateTiers":[],"interestRateTerms":"FIXED","interestSpread":0.0,"interestRateReviewCount":31,"interestRa

In [9]:
def addCurrentAccountToCreditLine():
    HEADERS = {'Content-Type': 'application/json'}
    PARAMS = {}
    r = POST('{{env4}}'+'/linesofcredit/{0}/savings/{1}'.format(CAID,CurrentAccID) , headers=HEADERS, params=PARAMS)
    print ("API status:")
    print(r.status_code)
    print("Now the JSON:")
    print(r.text)

In [239]:
addCurrentAccountToCreditLine()

API status:
200
Now the JSON:
{"encodedKey":"8a818eed6cd2de01016cd3d186145cf8","id":"RGBG661","accountHolderKey":"8a818f746cd2fea7016cd3ace4c74d28","accountHolderType":"CLIENT","name":"Current Account (with overdraft)","creationDate":"2019-08-27T16:03:39+0000","lastModifiedDate":"2019-08-27T16:03:39+0000","productTypeKey":"8a818f5f6cbe6621016cbf4f2db95756","lineOfCreditKey":"8a818f796cd39234016cd3d082dd0384","accountType":"CURRENT_ACCOUNT","accountState":"PENDING_APPROVAL","balance":"0","accruedInterest":"0","overdraftInterestAccrued":"0","technicalOverdraftInterestAccrued":"0","overdraftAmount":"0","technicalOverdraftAmount":"0","overdraftExpiryDate":"2020-05-02T00:00:00+0000","overdraftInterestSettings":{"interestSpread":"0","encodedKey":"8a818eed6cd2de01016cd3d186145cf9","interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateSource":"INDEX_INTEREST_RATE","interestRateReviewUnit":"DAYS","interestRateReviewCount":31,"interestRateTerms":"FIXED","interestRa

In [10]:
%%writefile APPROVEACC
{
  "action": "APPROVE",
  "notes": "Approved from the API"
}

Overwriting APPROVEACC


In [11]:
def approveDepositAccount():
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    url = "{{env4}}" + "/deposits/{0}:changeState".format(CurrentAccID)
    r = POST(url, headers=HEADERS, params=PARAMS, body="APPROVEACC")
    print(r.text)

In [241]:
approveDepositAccount()

{"encodedKey":"8a818eed6cd2de01016cd3d186145cf8","creationDate":"2019-08-27T18:03:39+02:00","lastModifiedDate":"2019-08-27T18:05:35+02:00","id":"RGBG661","name":"Current Account (with overdraft)","accountHolderType":"CLIENT","accountHolderKey":"8a818f746cd2fea7016cd3ace4c74d28","accountState":"APPROVED","productTypeKey":"8a818f5f6cbe6621016cbf4f2db95756","accountType":"CURRENT_ACCOUNT","creditArrangementKey":"8a818f796cd39234016cd3d082dd0384","approvedDate":"2019-08-27T18:05:36+02:00","currencyCode":"EUR","assignedBranchKey":"8a818f156ccf5fb1016cd2e8e4532b09","internalControls":{},"overdraftSettings":{"allowOverdraft":true,"overdraftLimit":0E-10,"overdraftExpiryDate":"2020-05-02T00:00:00+02:00"},"interestSettings":{"interestPaymentSettings":{"interestPaymentDates":[]}},"overdraftInterestSettings":{"interestRateSettings":{"encodedKey":"8a818eed6cd2de01016cd3d186145cf9","interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateTiers":[],"interestRateTerms":"FIX

## Create the CC bucket accounts


In [12]:
%%writefile CCACCTEMP
{
  "loanAmount": 30000.0,
  "loanName": "{{AccountName}}",
  "accountHolderKey": "{{CustomerID}}",
  "productTypeKey": "{{ProductKey}}",
  "accountHolderType": "CLIENT",
  "assignedBranchKey": "8a818f156ccf5fb1016cd2e8e4532b09",
  "interestFromArrearsAccrued": 0.0,
    "interestSettings": {
        "accrueInterestAfterMaturity": false,
        "interestApplicationMethod": "REPAYMENT_DUE_DATE",
        "interestBalanceCalculationMethod": "ONLY_PRINCIPAL",
        "interestCalculationMethod": "DECLINING_BALANCE",
        "interestChargeFrequency": "ANNUALIZED",
        "interestRateReviewCount": 31,
        "interestRateReviewUnit": "DAYS",
        "interestRateSource": "INDEX_INTEREST_RATE",
        "interestSpread": 0.0,
        "interestType": "SIMPLE_INTEREST"
    },
  "scheduleSettings": {
        "fixedDaysOfMonth": [
            1
        ],
        "gracePeriod": 0,
        "gracePeriodType": "NONE",
        "paymentPlan": [],
        "periodicPayment": 0.0,
        "principalRepaymentInterval": 1,
        "repaymentPeriodUnit": "DAYS",
        "repaymentScheduleMethod": "DYNAMIC",
        "scheduleDueDatesMethod": "FIXED_DAYS_OF_MONTH",
        "shortMonthHandlingMethod": "LAST_DAY_IN_MONTH"
    }
  
}

Overwriting CCACCTEMP


In [31]:
def createCreditCardBucket(cctype):
    global cc_account
    if cctype == "cash":
        productKey = "8a818f5f6cbe6621016cbf3cf8675424"
        accName = "CC - Cash"
    else:
        productKey = "8a818f5f6cbe6621016cbf6d66075e54"
        accName = "CC - Purchases"
    PLACEHOLDERS = {
        'CustomerID': newCustomerID,
        'ProductKey': productKey,
        'AccountName': accName
    }
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    r = POST('{{env4}}/loans', headers=HEADERS, params=PARAMS, body="CCACCTEMP", bodyparts=PLACEHOLDERS)
    print ("API status:")
    print(r.status_code)
    print("Now the JSON:")
    print(r.text)
    cc_account = r.json()['id']

In [246]:
createCreditCardBucket("purchase")

API status:
201
Now the JSON:
{"encodedKey":"8a818fd26cd2df22016cd3995fea4c35","id":"LVYN893","accountHolderType":"CLIENT","accountHolderKey":"8a818f746cd2fea7016cd3ace4c74d28","creationDate":"2019-08-27T18:11:01+02:00","lastModifiedDate":"2019-08-27T18:11:01+02:00","accountState":"PENDING_APPROVAL","productTypeKey":"8a818f5f6cbe6621016cbf6d66075e54","loanName":"CC - Purchases","loanAmount":30000.0,"paymentMethod":"HORIZONTAL","assignedBranchKey":"8a818f156ccf5fb1016cd2e8e4532b09","accruedInterest":0,"interestFromArrearsAccrued":0.0,"allowOffset":false,"arrearsTolerancePeriod":0,"accountArrearsSettings":{"encodedKey":"8a818fd26cd2df22016cd3d84276562e","toleranceCalculationMethod":"ARREARS_TOLERANCE_PERIOD","dateCalculationMethod":"ACCOUNT_FIRST_WENT_TO_ARREARS","nonWorkingDaysMethod":"EXCLUDED","tolerancePeriod":0},"latePaymentsRecalculationMethod":"OVERDUE_INSTALLMENTS_INCREASE","balances":{"redrawBalance":0,"principalDue":0,"principalPaid":0,"principalBalance":0,"interestDue":0,"inte

In [13]:
def addCCBucketToCreditLine():
    HEADERS = {'Content-Type': 'application/json'}
    PARAMS = {}
    r = POST('{{env4}}'+'/linesofcredit/{0}/loans/{1}'.format(CAID,cc_account) , headers=HEADERS, params=PARAMS)
    print ("API status:")
    print(r.status_code)
    print("Now the JSON:")
    print(r.text)

In [248]:
addCCBucketToCreditLine()

API status:
200
Now the JSON:
{"encodedKey":"8a818fd26cd2df22016cd3995fea4c35","id":"LVYN893","accountHolderKey":"8a818f746cd2fea7016cd3ace4c74d28","accountHolderType":"CLIENT","creationDate":"2019-08-27T16:11:01+0000","lastModifiedDate":"2019-08-27T16:11:01+0000","accountState":"PENDING_APPROVAL","productTypeKey":"8a818f5f6cbe6621016cbf6d66075e54","lineOfCreditKey":"8a818f796cd39234016cd3d082dd0384","loanName":"CC - Purchases","loanAmount":"30000","periodicPayment":"0","tranches":[],"principalDue":"0","principalPaid":"0","principalBalance":"0","redrawBalance":"0","interestDue":"0","interestPaid":"0","interestFromArrearsBalance":"0","interestFromArrearsDue":"0","interestFromArrearsPaid":"0","interestBalance":"0","feesDue":"0","feesPaid":"0","feesBalance":"0","penaltyDue":"0","penaltyPaid":"0","penaltyBalance":"0","scheduleDueDatesMethod":"FIXED_DAYS_OF_MONTH","prepaymentAcceptance":"ACCEPT_PREPAYMENTS","futurePaymentsAcceptance":"NO_FUTURE_PAYMENTS","hasCustomSchedule":false,"fixedDays

In [14]:
%%writefile APPROVEACC
{
  "action": "APPROVE",
  "notes": "Approved from the API"
}

Overwriting APPROVEACC


In [15]:
def approveLoanAccount():
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    url = "{{env4}}" + "/loans/{0}:changeState".format(cc_account)
    r = POST(url, headers=HEADERS, params=PARAMS, body="APPROVEACC")
    print(r.text)

In [251]:
approveLoanAccount()

{"encodedKey":"8a818fd26cd2df22016cd3995fea4c35","id":"LVYN893","accountHolderType":"CLIENT","accountHolderKey":"8a818f746cd2fea7016cd3ace4c74d28","creationDate":"2019-08-27T18:11:01+02:00","approvedDate":"2019-08-27T18:12:29+02:00","lastModifiedDate":"2019-08-27T18:12:28+02:00","accountState":"APPROVED","productTypeKey":"8a818f5f6cbe6621016cbf6d66075e54","creditArrangementKey":"8a818f796cd39234016cd3d082dd0384","loanName":"CC - Purchases","loanAmount":30000.0000000000,"paymentMethod":"HORIZONTAL","assignedBranchKey":"8a818f156ccf5fb1016cd2e8e4532b09","accruedInterest":0E-10,"interestFromArrearsAccrued":0E-10,"allowOffset":false,"arrearsTolerancePeriod":0,"accountArrearsSettings":{"encodedKey":"8a818fd26cd2df22016cd3d84276562e","toleranceCalculationMethod":"ARREARS_TOLERANCE_PERIOD","dateCalculationMethod":"ACCOUNT_FIRST_WENT_TO_ARREARS","nonWorkingDaysMethod":"EXCLUDED","tolerancePeriod":0},"latePaymentsRecalculationMethod":"OVERDUE_INSTALLMENTS_INCREASE","balances":{"redrawBalance":0

In [16]:
%%writefile INLOANACCTEMP
{
  "loanAmount": {{LoanAmount}},
  "loanName": "{{AccountName}}",
  "accountHolderKey": "{{CustomerID}}",
  "productTypeKey": "{{ProductKey}}",
  "accountHolderType": "CLIENT",
  "assignedBranchKey": "8a818f156ccf5fb1016cd2e8e4532b09",
  "interestFromArrearsAccrued": 0.0,
    "interestSettings": {
        "accrueInterestAfterMaturity": false,
        "interestApplicationMethod": "REPAYMENT_DUE_DATE",
        "interestBalanceCalculationMethod": "ONLY_PRINCIPAL",
        "interestCalculationMethod": "DECLINING_BALANCE_DISCOUNTED",
        "interestChargeFrequency": "ANNUALIZED",
        "interestRateReviewCount": 31,
        "interestRateReviewUnit": "DAYS",
        "interestRateSource": "INDEX_INTEREST_RATE",
        "interestSpread": 0.0,
        "interestType": "SIMPLE_INTEREST"
    },
  "scheduleSettings": {
        "gracePeriod": 0,
        "gracePeriodType": "NONE",
        "paymentPlan": [],
        "periodicPayment": 0.0,
        "principalRepaymentInterval": 1,
        "repaymentInstallments": 12,
        "repaymentPeriodCount": 1,
        "repaymentPeriodUnit": "MONTHS",
        "repaymentScheduleMethod": "DYNAMIC",
        "scheduleDueDatesMethod": "INTERVAL"
    }
  
}

Overwriting INLOANACCTEMP


In [439]:
%%writefile MERCHANTLOANACCTEMP
{
  "loanAmount": {{LoanAmount}},
  "loanName": "{{AccountName}}",
  "accountHolderKey": "{{CustomerID}}",
  "productTypeKey": "{{ProductKey}}",
  "accountHolderType": "CLIENT",
  "assignedBranchKey": "8a818f156ccf5fb1016cd2e8e4532b09",
  "interestFromArrearsAccrued": 0.0,
    "interestSettings": {
        "accrueInterestAfterMaturity": false,
        "interestApplicationMethod": "REPAYMENT_DUE_DATE",
        "interestBalanceCalculationMethod": "ONLY_PRINCIPAL",
        "interestCalculationMethod": "DECLINING_BALANCE",
        "interestChargeFrequency": "ANNUALIZED",
        "interestRate": 5.0,
        "interestRateSource": "FIXED_INTEREST_RATE",
        "interestType": "SIMPLE_INTEREST"
    },
  "scheduleSettings": {
        "fixedDaysOfMonth": [
            1
        ],
        "gracePeriod": 6,
        "gracePeriodType": "INTEREST_FORGIVENESS",
        "paymentPlan": [],
        "periodicPayment": 0.0,
        "principalRepaymentInterval": 1,
        "repaymentInstallments": 12,
        "repaymentPeriodCount": 0,
        "repaymentPeriodUnit": "DAYS",
        "repaymentScheduleMethod": "DYNAMIC",
        "scheduleDueDatesMethod": "FIXED_DAYS_OF_MONTH",
        "shortMonthHandlingMethod": "LAST_DAY_IN_MONTH"
    }
  
}

Overwriting MERCHANTLOANACCTEMP


In [17]:
def createInstalmentLoan(loanAmount):
    createLoan(loanAmount,"8a818f5f6cbe6621016cbf627bbc5af4", "Instalment Loan", "INLOANACCTEMP")
    
def createMerchantPromotionLoan(loanAmount):
    createLoan(loanAmount,"8a818f5f6cbe6621016cbf4b45be568b", "Merchant Promotion Loan", "MERCHANTLOANACCTEMP")
    
def createLoan(loanAmount, productKey, accName, tempFile):
    global cc_account
    
    PLACEHOLDERS = {
        'CustomerID': newCustomerID,
        'ProductKey': productKey,
        'AccountName': accName,
        'LoanAmount': loanAmount
    }
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    r = POST('{{env4}}/loans', headers=HEADERS, params=PARAMS, body=tempFile, bodyparts=PLACEHOLDERS)
    print ("API status:")
    print(r.status_code)
    print("Now the JSON:")
    print(r.text)
    cc_account = r.json()['id']
    addCCBucketToCreditLine()
    approveLoanAccount()
    print("***FINISHED")
    print(cc_account)
    return cc_account

In [443]:
createInstalmentLoan(2000)

API status:
201
Now the JSON:
{"encodedKey":"8a818f336cd48137016cd709a9583f3a","id":"RQNY670","accountHolderType":"CLIENT","accountHolderKey":"8a818e4f6cd67373016cd6fb4820089d","creationDate":"2019-08-28T09:06:18+02:00","lastModifiedDate":"2019-08-28T09:06:18+02:00","accountState":"PENDING_APPROVAL","productTypeKey":"8a818f5f6cbe6621016cbf627bbc5af4","loanName":"Instalment Loan","loanAmount":2000,"paymentMethod":"HORIZONTAL","assignedBranchKey":"8a818f156ccf5fb1016cd2e8e4532b09","accruedInterest":0,"interestFromArrearsAccrued":0.0,"allowOffset":false,"arrearsTolerancePeriod":0,"accountArrearsSettings":{"encodedKey":"8a818f336cd48137016cd709a9583f3d","toleranceCalculationMethod":"ARREARS_TOLERANCE_PERIOD","dateCalculationMethod":"ACCOUNT_FIRST_WENT_TO_ARREARS","nonWorkingDaysMethod":"EXCLUDED","tolerancePeriod":0},"latePaymentsRecalculationMethod":"OVERDUE_INSTALLMENTS_INCREASE","balances":{"redrawBalance":0,"principalDue":0,"principalPaid":0,"principalBalance":0,"interestDue":0,"intere

In [ ]:
{
    "accountArrearsSettings": {
        "dateCalculationMethod": "ACCOUNT_FIRST_WENT_TO_ARREARS",
        "encodedKey": "8a818f9c6cd48156016cd703677d3d81",
        "nonWorkingDaysMethod": "EXCLUDED",
        "toleranceCalculationMethod": "ARREARS_TOLERANCE_PERIOD",
        "tolerancePeriod": 0
    },
    "accountHolderKey": "8a818e4f6cd67373016cd6fb4820089d",
    "accountHolderType": "CLIENT",
    "accountState": "PENDING_APPROVAL",
    "accruedInterest": 0.0,
    "accruedPenalty": 0.0,
    "allowOffset": false,
    "arrearsTolerancePeriod": 0,
    "assets": [],
    "assignedBranchKey": "8a818f156ccf5fb1016cd2e8e4532b09",
    "balances": {
        "feesBalance": 0.0,
        "feesDue": 0.0,
        "feesPaid": 0.0,
        "holdBalance": 0.0,
        "interestBalance": 0.0,
        "interestDue": 0.0,
        "interestFromArrearsBalance": 0.0,
        "interestFromArrearsDue": 0.0,
        "interestFromArrearsPaid": 0.0,
        "interestPaid": 0.0,
        "penaltyBalance": 0.0,
        "penaltyDue": 0.0,
        "penaltyPaid": 0.0,
        "principalBalance": 0.0,
        "principalDue": 0.0,
        "principalPaid": 0.0,
        "redrawBalance": 0.0
    },
    "creationDate": "2019-08-28T08:57:00+02:00",
    "disbursementDetails": {
        "encodedKey": "8a818f9c6cd48156016cd70205e23d52",
        "fees": [],
        "transactionDetails": {
            "encodedKey": "8a818f9c6cd48156016cd70205e23d53",
            "internalTransfer": false,
            "transactionChannelId": "cash",
            "transactionChannelKey": "8a818e74677a2e9201677ec2b4c336a6"
        }
    },
    "encodedKey": "8a818f9c6cd48156016cd70205e23d51",
    "fundingSources": [],
    "futurePaymentsAcceptance": "NO_FUTURE_PAYMENTS",
    "guarantors": [],
    "id": "RWJI239",
    "interestFromArrearsAccrued": 0.0,
    "interestSettings": {
        "accrueInterestAfterMaturity": false,
        "accrueLateInterest": true,
        "interestApplicationMethod": "REPAYMENT_DUE_DATE",
        "interestBalanceCalculationMethod": "ONLY_PRINCIPAL",
        "interestCalculationMethod": "DECLINING_BALANCE",
        "interestChargeFrequency": "ANNUALIZED",
        "interestRate": 5.0,
        "interestRateSource": "FIXED_INTEREST_RATE",
        "interestType": "SIMPLE_INTEREST"
    },
    "lastModifiedDate": "2019-08-28T08:57:00+02:00",
    "latePaymentsRecalculationMethod": "OVERDUE_INSTALLMENTS_INCREASE",
    "loanAmount": 3000.0,
    "loanName": "Credit Card - Merchant Promotion ",
    "paymentMethod": "HORIZONTAL",
    "penaltySettings": {
        "loanPenaltyCalculationMethod": "NONE"
    },
    "prepaymentSettings": {
        "applyInterestOnPrepaymentMethod": "AUTOMATIC",
        "prepaymentRecalculationMethod": "NO_RECALCULATION",
        "principalPaidInstallmentStatus": "PARTIALLY_PAID"
    },
    "productTypeKey": "8a818f5f6cbe6621016cbf4b45be568b",
    "scheduleSettings": {
        "fixedDaysOfMonth": [
            1
        ],
        "gracePeriod": 6,
        "gracePeriodType": "INTEREST_FORGIVENESS",
        "hasCustomSchedule": false,
        "paymentPlan": [],
        "periodicPayment": 0.0,
        "principalRepaymentInterval": 1,
        "repaymentInstallments": 12,
        "repaymentPeriodCount": 0,
        "repaymentPeriodUnit": "DAYS",
        "repaymentScheduleMethod": "DYNAMIC",
        "scheduleDueDatesMethod": "FIXED_DAYS_OF_MONTH",
        "shortMonthHandlingMethod": "LAST_DAY_IN_MONTH"
    },
    "tranches": []
}

<a id="section1"></a>
[back](#sectionx)
## Add New Card Transaction

In [18]:
%%writefile NEWCARDTRANS
{
    "Amount": "10",
    "DebitCardAvailable": "1000",
    "DebitCardOnly": "F",
    "Merchant": "Starbucks",
    "POSCreditTypeChoice": "0",
    "PurchaseCategory": "Food+Drink",
    "transactionID": "{{TranID}}"
}

Overwriting NEWCARDTRANS


In [204]:
PLACEHOLDERS = {
    "TranID": getUUID()
}
HEADERS = {}
PARAMS = {}
r = POST('https://mpo-sandbox.mambucloud.com/api/1/json/public/18614/28388948acbf477da3d99dc3999e264ab745965a', 
         headers=HEADERS, params=PARAMS, body="NEWCARDTRANS", bodyparts=PLACEHOLDERS)
PRINT(r)

API status: 200
JSON Response:
{
    "ops": {
        "obj": "task",
        "obj_id": "5d654f0a09a8af0fc8002b63",
        "proc": "ok",
        "ref": null
    },
    "request_proc": "ok"
}


## Miscellaneous


In [59]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json'}
PARAMS = {'detailsLevel': 'FULL', 'paginationDetails': 'ON', 'limit': 10}
r = GET('{{env4}}/creditarrangements', headers=HEADERS, params=PARAMS)
PRINT(r)

API status: 200
JSON Response:
[
    {
        "_URepayOptions": {
            "AutoRepayMethod": "Direct-Debit",
            "PaymentDueDay": "1",
            "ShortMonthOption": "late"
        },
        "amount": 10000.0,
        "approvedDate": "2019-08-27T14:37:27+02:00",
        "availableCreditAmount": 10000.0,
        "consumedCreditAmount": 0.0,
        "creationDate": "2019-08-27T14:37:27+02:00",
        "encodedKey": "8a818f156ccf5fb1016cd2ef00482b47",
        "expireDate": "2020-09-10T00:00:00+02:00",
        "exposureLimitType": "APPROVED_AMOUNT",
        "holderKey": "8a818faf6cd2dc01016cd30cd28702e4",
        "holderType": "CLIENT",
        "id": "HGO606",
        "lastModifiedDate": "2019-08-27T14:37:27+02:00",
        "notes": "",
        "startDate": "2019-08-27T00:00:00+02:00",
        "state": "APPROVED"
    },
    {
        "amount": 10000.0,
        "approvedDate": "2019-08-23T18:00:29+02:00",
        "availableCreditAmount": 10000.0,
        "consumedCreditAmount

In [412]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json'}
# Parameters below set up pagination and only returns max 20
PARAMS = {'detailsLevel': 'FULL', 'paginationDetails': 'ON','limit': 20}
r = GET('{{env4}}/deposits/ENLI028', headers=HEADERS, params=PARAMS)
PRINT(r)

API status: 200
JSON Response:
{
    "accountHolderKey": "8a818f766cd3015d016cd3d8ac605e28",
    "accountHolderType": "CLIENT",
    "accountState": "PENDING_APPROVAL",
    "accountType": "CURRENT_ACCOUNT",
    "accruedAmounts": {
        "interestAccrued": 0.0,
        "overdraftInterestAccrued": 0.0,
        "technicalOverdraftInterestAccrued": 0.0
    },
    "assignedBranchKey": "8a818f156ccf5fb1016cd2e8e4532b09",
    "balances": {
        "availableBalance": 0.0,
        "feesDue": 0.0,
        "holdBalance": 0.0,
        "lockedBalance": 0.0,
        "overdraftAmount": 0.0,
        "overdraftInterestDue": 0.0,
        "technicalOverdraftAmount": 0.0,
        "technicalOverdraftInterestDue": 0.0,
        "totalBalance": 0
    },
    "creationDate": "2019-08-28T08:39:07+02:00",
    "currencyCode": "EUR",
    "encodedKey": "8a818f9c6cd48156016cd6ef16ec3c24",
    "id": "ENLI028",
    "interestSettings": {
        "interestPaymentSettings": {
            "interestPaymentDates": []
     

In [425]:

HEADERS = {'Accept': 'application/vnd.mambu.v2+json'}
# Parameters below set up pagination and only returns max 20
PARAMS = {'detailsLevel': 'FULL', 'paginationDetails': 'ON','limit': 20}
r = GET('{{env4}}/loans/RWJI239', headers=HEADERS, params=PARAMS)
PRINT(r)

API status: 200
JSON Response:
{
    "accountArrearsSettings": {
        "dateCalculationMethod": "ACCOUNT_FIRST_WENT_TO_ARREARS",
        "encodedKey": "8a818f9c6cd48156016cd703677d3d81",
        "nonWorkingDaysMethod": "EXCLUDED",
        "toleranceCalculationMethod": "ARREARS_TOLERANCE_PERIOD",
        "tolerancePeriod": 0
    },
    "accountHolderKey": "8a818e4f6cd67373016cd6fb4820089d",
    "accountHolderType": "CLIENT",
    "accountState": "PENDING_APPROVAL",
    "accruedInterest": 0.0,
    "accruedPenalty": 0.0,
    "allowOffset": false,
    "arrearsTolerancePeriod": 0,
    "assets": [],
    "assignedBranchKey": "8a818f156ccf5fb1016cd2e8e4532b09",
    "balances": {
        "feesBalance": 0.0,
        "feesDue": 0.0,
        "feesPaid": 0.0,
        "holdBalance": 0.0,
        "interestBalance": 0.0,
        "interestDue": 0.0,
        "interestFromArrearsBalance": 0.0,
        "interestFromArrearsDue": 0.0,
        "interestFromArrearsPaid": 0.0,
        "interestPaid": 0.0,
   

<a id="section3"></a>
[back](#sectionx)
## Get customer Accounts

In [19]:
%%writefile LOANSEARCH
{
  "filterConstraints":[
      {
        "filterSelection":"ACCOUNT_HOLDER_KEY",
        "filterElement":"EQUALS",
        "value":"{{CustomerID}}"
      },
      {
        "filterSelection":"PRODUCT_KEY",
        "filterElement":"EQUALS",
        "value":"{{ProductKey}}"
      }
   ]
}

Overwriting LOANSEARCH


In [20]:
def getCustomerAccount(accTy, productKey):
    if accTy == "loan":
        apiStr = '{{env4}}/loans/search'
    else:
        apiStr = '{{env4}}/savings/search'
    PLACEHOLDERS = {
            "CustomerID": newCustomerID,
            "ProductKey": productKey
        }
    HEADERS = {'Content-Type': 'application/json'}
    # Parameters below set up pagination and only returns max 20
    PARAMS = {}
    r = POST(apiStr, headers=HEADERS, params=PARAMS, body="LOANSEARCH", bodyparts=PLACEHOLDERS)
    resList = r.json()
    for resItem in resList:
        print(resItem["id"])
        print(resItem["accountHolderKey"])
        return resItem["id"]

In [21]:
def getCreditLine():
    HEADERS = {'Content-Type': 'application/json'}
    HEADERS = {}
    # Parameters below set up pagination and only returns max 20
    PARAMS = {}
    r = GET('{{env4}}'+'/clients/{0}/linesofcredit'.format(custShortID), headers=HEADERS, params=PARAMS)
    resList = r.json()
    for resItem in resList:
        print(resItem["id"])
        return resItem["id"]

In [461]:
# custShortID = 339063539
custShortID = "8a818e4f6cd67373016cd6fb4820089d"

In [463]:
getCreditLine()


KVL288


'KVL288'

In [351]:
'{{env4}}'+'//clients/{0}/linesofcredit?fullDetails=true'.format(newCustomerID)

'{{env4}}//clients/8a818efc6cd4813e016cd4ab3f3600b9/linesofcredit?fullDetails=true'

In [309]:
# CC Purchases Account
print("CC Purchases Account")
getCustomerAccount("loan","8a818f5f6cbe6621016cbf6d66075e54")

print("CC Cash Account")
getCustomerAccount("loan", "8a818f5f6cbe6621016cbf3cf8675424")

print("Current Account (with overdraft)")
getCustomerAccount("deposit", "8a818f5f6cbe6621016cbf4f2db95756")


CC Purchases Account
GNNW982
8a818f766cd3015d016cd3d8ac605e28
CC Cash Account
AUIN516
8a818f766cd3015d016cd3d8ac605e28
Current Account (with overdraft)
YMLH036
8a818f766cd3015d016cd3d8ac605e28


'YMLH036'

In [279]:
print(newCustomerID)

8a818f766cd3015d016cd3d8ac605e28


<a id="section4"></a>
[back](#sectionx)
## Disburse Loan

In [22]:
%%writefile DISBURSELOAN
{
  "type": "DISBURSEMENT",
  "amount": {{Amount}},
  "transactionChannelKey": "8a818f9c6cd48156016cd71cc9874322"
}

Overwriting DISBURSELOAN


In [23]:
def disburseLoan(loanID, amount):
    
    PLACEHOLDERS = {
            "CustomerID": newCustomerID,
            "Amount": amount
        }
    HEADERS = {'Content-Type': 'application/json'}
    # Parameters below set up pagination and only returns max 20
    PARAMS = {}
    r = POST('{{env4}}'+'/loans/{0}/transactions'.format(loanID), headers=HEADERS, params=PARAMS, body="DISBURSELOAN", bodyparts=PLACEHOLDERS)
    PRINT(r)
    print("***FINISH***")
    tranID = r.json()["transactionId"]
    print(tranID)
    #resList = r.json()
    #for resItem in resList:
    #    print(resItem["id"])
    #    print(resItem["accountHolderKey"])
    #    return resItem["id"]

In [453]:
disburseLoan("HHAE369", 55)

API status: 201
JSON Response:
{
    "advancePosition": "0",
    "amount": "55",
    "arrearsPosition": "0",
    "balance": "779",
    "bookingDate": "2019-08-28T09:32:24+0000",
    "branchKey": "8a818f156ccf5fb1016cd2e8e4532b09",
    "creationDate": "2019-08-28T07:32:25+0000",
    "deferredInterestAmount": "0",
    "deferredTaxOnInterestAmount": "0",
    "details": {
        "encodedKey": "8a818e4f6cd67373016cd70befba0b65",
        "internalTransfer": false,
        "transactionChannel": {
            "activated": true,
            "creationDate": "2018-12-05T15:25:49+0000",
            "encodedKey": "8a818e74677a2e9201677ec2b4c336a6",
            "id": "cash",
            "index": 0,
            "name": "Cash",
            "savingsConstraints": {
                "encodedKey": "8a818e74677a2e9201677ec2b4c336a8",
                "filterConstraints": []
            },
            "savingsConstraintsUsage": "UNCONSTRAINED_USAGE",
            "usageRights": {
                "encodedKey":

In [24]:
%%writefile PATCHLOANCUSTOMFIELD
{
  "value": "{{Value}}"
}

Writing PATCHLOANCUSTOMFIELD


In [25]:
def patchLoanCF(loanID, transID, field, val):
    
    PLACEHOLDERS = {
            "Value": val
        }
    HEADERS = {'Content-Type': 'application/json'}
    # Parameters below set up pagination and only returns max 20
    PARAMS = {}
    r = PATCH('{{env4}}'+'/loans/{0}/transactions/{1}/custominformation/{3}'.format(loanID, transID, field), headers=HEADERS, params=PARAMS, body="PATCHLOANCUSTOMFIELD", bodyparts=PLACEHOLDERS)
    PRINT(r)

In [ ]:
patchLoanCF("HHAE369",528)

In [450]:
    HEADERS = {'Content-Type': 'application/json'}
    # Parameters below set up pagination and only returns max 20
    PARAMS = {}
    r = GET('{{env4}}'+'/loans/HHAE369/transactions/529', headers=HEADERS, params=PARAMS)
    PRINT(r)

API status: 200
JSON Response:
[
    {
        "advancePosition": "0",
        "amount": "654",
        "arrearsPosition": "0",
        "balance": "724",
        "bookingDate": "2019-08-28T09:27:57+0000",
        "branchKey": "8a818f156ccf5fb1016cd2e8e4532b09",
        "comment": "",
        "creationDate": "2019-08-28T07:27:58+0000",
        "customInformation": [
            {
                "customField": {
                    "amounts": {},
                    "creationDate": "2019-08-28T07:24:17+0000",
                    "customFieldSelectionOptions": [],
                    "customFieldSet": {
                        "createdDate": "2019-08-28T07:22:59+0000",
                        "encodedKey": "8a818f9c6cd48156016cd70368ef3d92",
                        "id": "_CardTransaction",
                        "indexInList": 2,
                        "lastModifiedDate": "2019-08-28T07:22:59+0000",
                        "name": "CardTransaction",
                        "notes": ""

In [394]:
disburseLoan("LBTW351", 2000)


API status: 201
JSON Response:
{
    "advancePosition": "0",
    "amount": "2000",
    "arrearsPosition": "0",
    "balance": "2000",
    "bookingDate": "2019-08-27T22:47:43+0000",
    "branchKey": "8a818f156ccf5fb1016cd2e8e4532b09",
    "creationDate": "2019-08-27T20:47:44+0000",
    "deferredInterestAmount": "0",
    "deferredTaxOnInterestAmount": "0",
    "details": {
        "encodedKey": "8a818efc6cd4813e016cd4d59a35031c",
        "internalTransfer": false,
        "transactionChannel": {
            "activated": true,
            "creationDate": "2018-12-05T15:25:49+0000",
            "encodedKey": "8a818e74677a2e9201677ec2b4c336a6",
            "id": "cash",
            "index": 0,
            "name": "Cash",
            "savingsConstraints": {
                "encodedKey": "8a818e74677a2e9201677ec2b4c336a8",
                "filterConstraints": []
            },
            "savingsConstraintsUsage": "UNCONSTRAINED_USAGE",
            "usageRights": {
                "encodedKe

<a id="section2"></a>
[back](#sectionx)
## Create ALL in 1 go

In [26]:
def createALL(fName, lName):
    #createNewCustomerTest(fName, lName)
    createNewCustomerDemo(fName, lName)
    createNewCreditArrangement()
    createTEMPCardDDAAccount()
    addCurrentAccountToCreditLine()
    approveDepositAccount()
    createCurrentAccount()
    addCurrentAccountToCreditLine()
    approveDepositAccount()
    createCreditCardBucket("cash")
    addCCBucketToCreditLine()
    approveLoanAccount()
    createCreditCardBucket("purchase")
    addCCBucketToCreditLine()
    approveLoanAccount()

In [32]:
createALL("Test", "Cust04043")
print("**********Finished ***********************")
print("")
print(custShortID)

API status:
201
Now the JSON:
{"encodedKey":"8a818eac71361c2f017144c65e663d81","id":"585420596","state":"INACTIVE","creationDate":"2020-04-04T12:39:43+02:00","lastModifiedDate":"2020-04-04T12:39:43+02:00","approvedDate":"2020-04-04T12:39:43+02:00","firstName":"Test","lastName":"Cust04043","preferredLanguage":"ENGLISH","gender":"MALE","notes":"Some Notes on this person","assignedBranchKey":"8a818f5f6cbe6621016cbf217c9e5060","clientRoleKey":"8a818e74677a2e9201677ec2b4c336aa","loanCycle":0,"groupLoanCycle":0,"addresses":[{"encodedKey":"8a818eac71361c2f017144c65e663d82","parentKey":"8a818eac71361c2f017144c65e663d81","city":"Liverpool","country":"UK","indexInList":0}],"idDocuments":[]}
API status:
201
Now the JSON:
{"encodedKey":"8a818fe371407db0017144c665ef0a25","id":"EPN886","holderType":"CLIENT","holderKey":"8a818eac71361c2f017144c65e663d81","amount":10000.0,"availableCreditAmount":10000.0,"consumedCreditAmount":0,"state":"APPROVED","creationDate":"2020-04-04T12:39:45+02:00","lastModifie

{"encodedKey":"8a818ea6713625ef017144c53924183e","creationDate":"2020-04-04T12:39:57+02:00","lastModifiedDate":"2020-04-04T12:40:02+02:00","id":"WNTP727","name":"Current Account (with overdraft)","accountHolderType":"CLIENT","accountHolderKey":"8a818eac71361c2f017144c65e663d81","accountState":"APPROVED","productTypeKey":"8a818f5f6cbe6621016cbf4f2db95756","accountType":"CURRENT_ACCOUNT","creditArrangementKey":"8a818fe371407db0017144c665ef0a25","approvedDate":"2020-04-04T12:40:02+02:00","currencyCode":"EUR","assignedBranchKey":"8a818f5f6cbe6621016cbf217c9e5060","internalControls":{},"overdraftSettings":{"allowOverdraft":true,"overdraftLimit":0,"overdraftExpiryDate":"2020-05-02T00:00:00+02:00"},"interestSettings":{"interestPaymentSettings":{"interestPaymentDates":[]}},"overdraftInterestSettings":{"interestRateSettings":{"encodedKey":"8a818ea6713625ef017144c53924183f","interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateTiers":[],"interestRateTerms":"FIXED",

API status:
201
Now the JSON:
{"encodedKey":"8a818fcb71362638017144c6cd102da0","id":"FAXV867","accountHolderType":"CLIENT","accountHolderKey":"8a818eac71361c2f017144c65e663d81","creationDate":"2020-04-04T12:40:11+02:00","lastModifiedDate":"2020-04-04T12:40:11+02:00","accountState":"PENDING_APPROVAL","productTypeKey":"8a818f5f6cbe6621016cbf6d66075e54","loanName":"CC - Purchases","loanAmount":30000.0,"paymentMethod":"HORIZONTAL","assignedBranchKey":"8a818f156ccf5fb1016cd2e8e4532b09","accruedInterest":0,"interestFromArrearsAccrued":0,"allowOffset":false,"arrearsTolerancePeriod":0,"accountArrearsSettings":{"encodedKey":"8a818fcb71362638017144c6cd102da3","toleranceCalculationMethod":"ARREARS_TOLERANCE_PERIOD","dateCalculationMethod":"ACCOUNT_FIRST_WENT_TO_ARREARS","nonWorkingDaysMethod":"EXCLUDED","tolerancePeriod":0},"latePaymentsRecalculationMethod":"OVERDUE_INSTALLMENTS_INCREASE","balances":{"redrawBalance":0,"principalDue":0,"principalPaid":0,"principalBalance":0,"interestDue":0,"intere

In [ ]:

createInstalmentLoan(4000)



In [ ]:
disburseLoan("NELC968", 1000)

In [343]:
print(custShortID)

519155005
